# Analise usando Querys

In [5]:
import numpy as np
import pandas as pd
import missingno as mno
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact_manual
from geopy import Nominatim

from sklearn.preprocessing import MinMaxScaler

In [2]:
city = pd.read_csv('Dados/city_clear.csv')
city.head()

,Unnamed: 0,Location,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Beer (0.5 liter draught),Imported Beer (0.33 liter bottle),Coke/Pepsi (0.33 liter bottle),Water (0.33 liter bottle),"Milk (regular), (1 liter)",...,latitude,longitude,Food,Travel,Living,Lifestyle,Education,Income,country,Country
0,0,"Saint Petersburg, Russia",7.34,29.35,4.40,2.20,2.20,0.76,0.53,0.98,...,59.938732,30.316229,2,8,5020,34,2900,328,"['Saint Petersburg', 'Russia']",Russia
1,1,"Istanbul, Turkey",4.58,15.28,3.82,3.06,3.06,0.64,0.24,0.71,...,41.009633,28.965165,2,6,5409,23,3594,228,"['Istanbul', 'Turkey']",Turkey
2,2,"Izmir, Turkey",3.06,12.22,3.06,2.29,2.75,0.61,0.22,0.65,...,38.414733,27.143412,2,5,5297,19,2580,207,"['Izmir', 'Turkey']",Turkey
3,3,"Helsinki, Finland",12.00,65.00,8.00,6.50,6.75,2.66,1.89,0.96,...,60.167488,24.942747,4,20,6985,44,996,1192,"['Helsinki', 'Finland']",Finland
4,4,"Chisinau, Moldova",4.67,20.74,4.15,1.04,1.43,0.64,0.44,0.68,...,47.024512,28.832292,2,2,4040,32,1445,148,"['Chisinau', 'Moldova']",Moldova


In [3]:
# Comparação com as cidades mais famosas
city[(city['Country'] == 'United Kingdom') | (city['Country'] == 'Australia') | 
     (city['Country'] == 'Germany') | (city['Country'] == 'China') | 
    (city['Country'] == 'Russia')][['Location','Food', 
                'Travel','Living','Lifestyle','Education',
                    'Income']].set_index('Location').sort_values(by = 'Income',
                                ascending = False).style.background_gradient(cmap = 'copper')

,Food,Travel,Living,Lifestyle,Education,Income
Location,,,,,,
"Brisbane, Australia",5,22,4644,39,4910,1517
"Sydney, Australia",5,29,5781,42,6661,1511
"Melbourne, Australia",4,21,5218,39,7062,1398
"London, United Kingdom",4,37,9069,42,11423,1380
"Adelaide, Australia",4,16,4309,34,4606,1329
"Munich, Germany",4,17,7445,44,5867,1315
"Frankfurt, Germany",3,22,6871,42,6502,1310
"Hamburg, Germany",3,21,5940,44,7085,1267
"Perth, Australia",4,20,4709,39,6358,1249


In [6]:
# As cidades mais caras
x = city[['Food','Travel','Living','Lifestyle','Education','Income']]
mm = MinMaxScaler()
data = mm.fit_transform(x)
data = pd.DataFrame(data)
data.columns = x.columns
data.head()

,Food,Travel,Living,Lifestyle,Education,Income
0,0.142857,0.194444,0.139139,0.372549,0.139049,0.074890
1,0.142857,0.138889,0.161588,0.156863,0.177913,0.043424
2,0.142857,0.111111,0.155125,0.078431,0.121129,0.036816
3,0.428571,0.527778,0.252539,0.568627,0.032424,0.346759
4,0.142857,0.027778,0.082583,0.333333,0.057568,0.018250


In [13]:
data['Total Score'] = (data['Food'] + data['Travel'] + data['Living'] + 
                       data['Lifestyle'] + data['Education'] + data['Income'])/6

cities = city[['Country']]
data = pd.concat([data, cities], axis = 1)

print("Os lugares mais caros")
data[['Country','Total Score']].sort_values(by = 'Total Score', ascending = False).head(10)

Os lugares mais caros


,Country,Country,Country,Country,Total Score
144,Switzerland,Switzerland,Switzerland,Switzerland,0.790375
78,NY,NY,NY,NY,0.670422
100,CA,CA,CA,CA,0.660000
39,Singapore,Singapore,Singapore,Singapore,0.576437
70,United Kingdom,United Kingdom,United Kingdom,United Kingdom,0.558841
128,Iceland,Iceland,Iceland,Iceland,0.557198
102,WA,WA,WA,WA,0.539670
50,Norway,Norway,Norway,Norway,0.534259
47,MA,MA,MA,MA,0.525962
71,CA,CA,CA,CA,0.487471


In [15]:
# O lugar mais barato
print("Cheapest Places in the World\n")
data[['Country','Total Score']].sort_values(by = 'Total Score', ascending = True).head(10)

Cheapest Places in the World



,Country,Country,Country,Country,Total Score
123,India,India,India,India,0.042501
106,India,India,India,India,0.043629
112,India,India,India,India,0.056045
63,Georgia,Georgia,Georgia,Georgia,0.057976
118,India,India,India,India,0.060394
81,India,India,India,India,0.063713
130,India,India,India,India,0.069141
82,Sri Lanka,Sri Lanka,Sri Lanka,Sri Lanka,0.069626
114,India,India,India,India,0.070959
127,India,India,India,India,0.071562


In [17]:
# Verificando o preço de alimentos basicos
x = city[['Country','Milk (regular), (1 liter)',
         'Eggs (regular) (12)', 'Loaf of Fresh White Bread (500g)',
         'Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)',
         'Taxi 1km (Normal Tariff)',
         'Basic (Electricity, Heating, Cooling, Water, Garbage) for 85m2 Apartment',
          ]]

x = x.rename(columns = {'Milk (regular), (1 liter)':'Milk','Eggs (regular) (12)':'Eggs', 
                       'Loaf of Fresh White Bread (500g)': 'Bread',
                       'Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)': 'Internet',
                       'Taxi 1km (Normal Tariff)':'Taxi Travel',
                       'Basic (Electricity, Heating, Cooling, Water, Garbage) for 85m2 Apartment':'Electricity and Water'})
x.head()

,Country,Milk,Eggs,Bread,Internet,Taxi Travel,Electricity and Water
0,Russia,0.98,1.18,0.71,6.96,0.26,102.17
1,Turkey,0.71,1.62,0.36,14.20,0.47,59.33
2,Turkey,0.65,1.51,0.38,12.89,0.57,51.07
3,Finland,0.96,2.02,2.27,22.31,1.00,82.66
4,Moldova,0.68,1.11,0.33,8.58,0.18,113.46


In [18]:
x[['Milk','Bread','Eggs','Internet','Taxi Travel','Electricity and Water']].astype('float').describe()

,Milk,Bread,Eggs,Internet,Taxi Travel,Electricity and Water
count,160.000000,160.000000,160.000000,160.000000,160.000000,160.000000
mean,0.998938,1.197875,1.902812,29.660875,0.922250,107.106125
std,0.391720,0.760670,0.752520,18.908249,0.709011,51.553830
min,0.390000,0.100000,0.750000,4.440000,0.140000,18.560000
25%,0.710000,0.555000,1.377500,12.832500,0.405000,63.860000
50%,0.895000,1.020000,1.850000,26.615000,0.630000,102.465000
75%,1.170000,1.690000,2.352500,43.317500,1.350000,145.707500
max,2.640000,3.330000,5.330000,93.290000,4.160000,265.520000


In [21]:
plt.figure(figsize = (10, 3))
# Preço do Leite
print(x[x['Milk'] > 1.17][['Country','Milk']].sort_values(by = 'Milk',
                                    ascending = False).head(5).set_index('Country'))

print('\n')
# Preço do Pão
print(x[x['Bread'] > 1.69][['Country','Bread']].sort_values(by = 'Bread',
                                    ascending = False).head(5).set_index('Country'))

print('\n')
# Preço do Ovo
print(x[x['Eggs'] > 2.35][['Country','Eggs']].sort_values(by = 'Eggs',
                                    ascending = False).head(5).set_index('Country'))

print('\n')
# Preço da Internet
print(x[x['Internet'] > 43.37][['Country','Internet']].sort_values(by = 'Internet',
                                    ascending = False).head(5).set_index('Country'))

print('\n')
# Preço do Taxi
print(x[x['Taxi Travel'] > 1.35][['Country','Taxi Travel']].sort_values(by = 'Taxi Travel',
                                    ascending = False).head(5).set_index('Country'))

print('\n')
# Preço da Agua e Luz
print(x[x['Electricity and Water'] > 145.7][['Country',
                        'Electricity and Water']].sort_values(by = 'Electricity and Water',
                                    ascending = False).head(5).set_index('Country'));

             Milk
Country          
Taiwan       2.64
Hong Kong    2.54
China        2.39
Singapore    2.04
South Korea  1.95


         Bread
Country       
NY        3.33
CA        3.27
CA        3.12
CA        2.99
Norway    2.92


             Eggs
Country          
Switzerland  5.33
Iceland      4.80
Norway       3.79
France       3.40
Israel       3.32


                      Internet
Country                       
United Arab Emirates     93.29
United Arab Emirates     90.42
Qatar                    78.31
AZ                       67.23
Costa Rica               65.30


                    Taxi Travel
Country                        
Switzerland                4.16
Japan                      3.44
Netherlands                3.00
United Kingdom             2.97
Dominican Republic         2.55


          Electricity and Water
Country                        
Germany                  265.52
Germany                  242.66
Germany                  232.62
Germany                  231.80


<Figure size 720x216 with 0 Axes>